In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Firefox
import selenium.webdriver.firefox.options as fOptions
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
from tabulate import tabulate
import json
from win32process import CREATE_NO_WINDOW
import time
import numpy as np
from selenium.common import exceptions
import json
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent
import pandas as pd
from freesms import FreeClient



# returns Firefox webdriver
def getFirefoxDriver():
    
    return Firefox()

# returns a chrome webdriver with added options
# careful: dnb.com does not build webelements in headless mode
def getChromeDriver():
    
    chrome_options = Options()
    ua = UserAgent()
    userAgent = ua.chrome
    chrome_options.add_argument('user-agent='+userAgent)
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--window-size=1920,1080")
    #chrome_options.add_argument("--incognito")
    #chrome_options.headless = True
    chrome_options.add_argument("--headless")

    #chrome_options.add_argument("--hidecommandpromptwindow")
    chrome_driver = os.getcwd() +"\\chromedriver.exe"
    args = ["hide_console", ]
    
    return webdriver.Chrome(options=chrome_options, executable_path=chrome_driver, service_args=args)


# this method scrapes dnb.com from the start page, meaning that the webdriver puts in the name of the company to search
# for "manually", to get redirected to the search result site. It deliberately works slowly to avoid detection of 
# automation (not sure if that's necessary). Once the company name is put in and RETURN is sent, dnb.com denies access
# to the server
def scrape1(company): 
    driver = getFirefoxDriver() 
    driver.get("https://www.dnb.com/")
    time.sleep(1)
    search_button = driver.find_element_by_xpath("/html/body/div[2]/header/div[3]/div/div/div[2]/div[1]/button")
    time.sleep(1)
    search_button.click()
    search_bar = driver.find_element_by_xpath("/html/body/div[2]/div[2]/div[2]/div/div/div/div/div[2]/input")
    time.sleep(1)
    
    for character in company: 
        search_bar.send_keys(character)
        time.sleep(0.2)
    time.sleep(1)
    search_bar.send_keys(Keys.RETURN)

# this method builds the search query url for a specific company on dnb.com, avoiding the manual search for the company.
# Again, clicking on the company in the search result denies access to the server. 
def scrape2(company):
    driver = getChromeDriver()
    driver.get("https://www.dnb.com/business-directory/company-search.html?term="+ company +"&page=1")
    
    timeout = 5
    
    try:
        element_present = EC.presence_of_element_located((By.ID, 'company_results'))
        WebDriverWait(driver, timeout).until(element_present)
        time.sleep(1)
    except TimeoutException as e:
        return("failed")
    
    try:
        no_result = driver.find_element_by_id("no_results_message")
        return("not in db")
    except exceptions.NoSuchElementException as e: 
        try:
            results = driver.find_elements_by_class_name('row.search_result')
            for result in results: 
                primary_name =result.find_element_by_class_name('primary_name')
                if primary_name.text.lower() == company.lower():
                    a = primary_name.find_elements_by_tag_name("a")[0]
                    href = a.get_attribute("href")
                    driver.quit()
                    return href
                    break;
            return ("not in db")
            
        except exceptions.NoSuchElementException as e: 
                return "failed"
        
    


def scrape3(company_link):
    driver = getChromeDriver()
    driver.get(company_link)
    
    try:
        n_employees = driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div/div[6]/div/div/div/div/div[2]/ul/li[1]/span[2]").text
    except exceptions.NoSuchElementException as e: 
        n_employees = "missing"
    
    try:
        n_sales = driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div/div[6]/div/div/div/div/div[2]/ul/li[2]/span[2]").text
    except exceptions.NoSuchElementException as e: 
        n_sales = "missing"
    
    try:
        location = driver.find_element_by_class_name("company_postal").text
    except exceptions.NoSuchElementException as e: 
        location = "missing"
    
    driver.quit()
    return n_employees,n_sales,location


companies = []
with open('company_names_IT.json','r') as file: 
        name_list = json.load(file)
for name in name_list:
    if name is None: 
        continue
    print(name)
    company_link = scrape2(name)
    print(company_link)
    while company_link == "failed":
        company_link = scrape2(name)
    if company_link == "not in db":
        continue
        
    n_employees, n_sales,location = scrape3(company_link)
    plz, city = location.split(",")
    companies.append([name,n_employees,n_sales,plz,city])
    #print(companies)    

with open('IT/company_attributes.json','w') as f:
   json.dump(list(companies),f)


vleet GmbH
https://www.dnb.com/business-directory/company-profiles.vleet_gmbh.580755a92e8afd37501f5104b64e0ddb.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg']]
DSV Segel-Service GmbH
not in db
Emmet Software Labs GmbH & Co. KG
https://www.dnb.com/business-directory/company-profiles.emmet_software_labs_gmbh__co_kg.49da02ed94594ea99c4c4a67c2fc264b.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg'], ['Emmet Software Labs GmbH & Co. KG', '23', '0.73', '32052', ' Herford']]
Mittelständisches Systemhaus im Westen Münchens
not in db
kimeta GmbH
https://www.dnb.com/business-directory/company-profiles.kimeta_gmbh.6def6502ce41c4a668edce072a72a0b5.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg'], ['Emmet Software Labs GmbH & Co. KG', '23', '0.73', '32052', ' Herford'], ['kimeta GmbH', '61', '8.64', '64295', ' Darmstadt']]
DataCo GmbH
https://www.dnb.com/business-directory/company-profiles.dataco_gmbh.2cb8deaf2f7ea86ae75b38113b4ca3af.html
[['vleet GmbH', '3', '0.3', '82152', ' Planeg

not in db
INCONTROL Simulation Solutions
not in db
Bitronic Hardware Service GmbH (Sömmerda)
not in db
Leapsome
not in db
Albacon Systemhaus GmbH
https://www.dnb.com/business-directory/company-profiles.albacon_systemhaus_gmbh.6ad01a4200a8090df338be2fa9ef3a9a.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg'], ['Emmet Software Labs GmbH & Co. KG', '23', '0.73', '32052', ' Herford'], ['kimeta GmbH', '61', '8.64', '64295', ' Darmstadt'], ['DataCo GmbH', '100', '5.9', '80335', ' MÜNchen'], ['Baremos IT Dienstleistungs GmbH', '3', '3', '10407', ' Berlin'], ['711media websolutions GmbH', '20', '1.97', '70178', ' Stuttgart'], ['clickworker GmbH', '20', 'missing', '45149', ' Essen'], ['AMPADA GmbH', '9', '1.32', '50765', ' KÖLn'], ['tradefinity GmbH', '28', 'DEC', '25474', ' Ellerbek'], ['UFIT AG', '13', '1.25', '67141', ' Neuhofen'], ['FORCAM GmbH', '95', '14.04', '88214', ' Ravensburg'], ['Müller und Richter Informationssysteme GmbH', '5', '0.59', '63571', ' Gelnhausen'], ['OTRS AG', '63'

not in db
Honicon GmbH
https://www.dnb.com/business-directory/company-profiles.honicon_gmbh.2dbdeb24a71983fa7a2680898c34939e.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg'], ['Emmet Software Labs GmbH & Co. KG', '23', '0.73', '32052', ' Herford'], ['kimeta GmbH', '61', '8.64', '64295', ' Darmstadt'], ['DataCo GmbH', '100', '5.9', '80335', ' MÜNchen'], ['Baremos IT Dienstleistungs GmbH', '3', '3', '10407', ' Berlin'], ['711media websolutions GmbH', '20', '1.97', '70178', ' Stuttgart'], ['clickworker GmbH', '20', 'missing', '45149', ' Essen'], ['AMPADA GmbH', '9', '1.32', '50765', ' KÖLn'], ['tradefinity GmbH', '28', 'DEC', '25474', ' Ellerbek'], ['UFIT AG', '13', '1.25', '67141', ' Neuhofen'], ['FORCAM GmbH', '95', '14.04', '88214', ' Ravensburg'], ['Müller und Richter Informationssysteme GmbH', '5', '0.59', '63571', ' Gelnhausen'], ['OTRS AG', '63', '8', '61440', ' Oberursel (Taunus)'], ['Conclusive Consulting GmbH', '8', '0.44', '64653', ' Lorsch'], ['Albacon Systemhaus GmbH', '

https://www.dnb.com/business-directory/company-profiles.siteimprove_gmbh.a1eb76e28b0d7df6cd60c669082dd33d.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg'], ['Emmet Software Labs GmbH & Co. KG', '23', '0.73', '32052', ' Herford'], ['kimeta GmbH', '61', '8.64', '64295', ' Darmstadt'], ['DataCo GmbH', '100', '5.9', '80335', ' MÜNchen'], ['Baremos IT Dienstleistungs GmbH', '3', '3', '10407', ' Berlin'], ['711media websolutions GmbH', '20', '1.97', '70178', ' Stuttgart'], ['clickworker GmbH', '20', 'missing', '45149', ' Essen'], ['AMPADA GmbH', '9', '1.32', '50765', ' KÖLn'], ['tradefinity GmbH', '28', 'DEC', '25474', ' Ellerbek'], ['UFIT AG', '13', '1.25', '67141', ' Neuhofen'], ['FORCAM GmbH', '95', '14.04', '88214', ' Ravensburg'], ['Müller und Richter Informationssysteme GmbH', '5', '0.59', '63571', ' Gelnhausen'], ['OTRS AG', '63', '8', '61440', ' Oberursel (Taunus)'], ['Conclusive Consulting GmbH', '8', '0.44', '64653', ' Lorsch'], ['Albacon Systemhaus GmbH', '6', '3.89', '18209'

not in db
Hitachi Solutions Germany GmbH
https://www.dnb.com/business-directory/company-profiles.hitachi_solutions_germany_gmbh.3576f14c46ff206b2890247f5ce89eb1.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg'], ['Emmet Software Labs GmbH & Co. KG', '23', '0.73', '32052', ' Herford'], ['kimeta GmbH', '61', '8.64', '64295', ' Darmstadt'], ['DataCo GmbH', '100', '5.9', '80335', ' MÜNchen'], ['Baremos IT Dienstleistungs GmbH', '3', '3', '10407', ' Berlin'], ['711media websolutions GmbH', '20', '1.97', '70178', ' Stuttgart'], ['clickworker GmbH', '20', 'missing', '45149', ' Essen'], ['AMPADA GmbH', '9', '1.32', '50765', ' KÖLn'], ['tradefinity GmbH', '28', 'DEC', '25474', ' Ellerbek'], ['UFIT AG', '13', '1.25', '67141', ' Neuhofen'], ['FORCAM GmbH', '95', '14.04', '88214', ' Ravensburg'], ['Müller und Richter Informationssysteme GmbH', '5', '0.59', '63571', ' Gelnhausen'], ['OTRS AG', '63', '8', '61440', ' Oberursel (Taunus)'], ['Conclusive Consulting GmbH', '8', '0.44', '64653', ' Lor

...

not in db
POWERCASE Informatik GmbH
not in db
Gruner + Jahr GmbH
https://www.dnb.com/business-directory/company-profiles.gruner__jahr_gmbh.b9ae0d0e8d687374e759b951a505857b.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg'], ['Emmet Software Labs GmbH & Co. KG', '23', '0.73', '32052', ' Herford'], ['kimeta GmbH', '61', '8.64', '64295', ' Darmstadt'], ['DataCo GmbH', '100', '5.9', '80335', ' MÜNchen'], ['Baremos IT Dienstleistungs GmbH', '3', '3', '10407', ' Berlin'], ['711media websolutions GmbH', '20', '1.97', '70178', ' Stuttgart'], ['clickworker GmbH', '20', 'missing', '45149', ' Essen'], ['AMPADA GmbH', '9', '1.32', '50765', ' KÖLn'], ['tradefinity GmbH', '28', 'DEC', '25474', ' Ellerbek'], ['UFIT AG', '13', '1.25', '67141', ' Neuhofen'], ['FORCAM GmbH', '95', '14.04', '88214', ' Ravensburg'], ['Müller und Richter Informationssysteme GmbH', '5', '0.59', '63571', ' Gelnhausen'], ['OTRS AG', '63', '8', '61440', ' Oberursel (Taunus)'], ['Conclusive Consulting GmbH', '8', '0.44', '64

https://www.dnb.com/business-directory/company-profiles.visigate_software_gmbh.293adf096f627d0e3ef58ec109558c4e.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg'], ['Emmet Software Labs GmbH & Co. KG', '23', '0.73', '32052', ' Herford'], ['kimeta GmbH', '61', '8.64', '64295', ' Darmstadt'], ['DataCo GmbH', '100', '5.9', '80335', ' MÜNchen'], ['Baremos IT Dienstleistungs GmbH', '3', '3', '10407', ' Berlin'], ['711media websolutions GmbH', '20', '1.97', '70178', ' Stuttgart'], ['clickworker GmbH', '20', 'missing', '45149', ' Essen'], ['AMPADA GmbH', '9', '1.32', '50765', ' KÖLn'], ['tradefinity GmbH', '28', 'DEC', '25474', ' Ellerbek'], ['UFIT AG', '13', '1.25', '67141', ' Neuhofen'], ['FORCAM GmbH', '95', '14.04', '88214', ' Ravensburg'], ['Müller und Richter Informationssysteme GmbH', '5', '0.59', '63571', ' Gelnhausen'], ['OTRS AG', '63', '8', '61440', ' Oberursel (Taunus)'], ['Conclusive Consulting GmbH', '8', '0.44', '64653', ' Lorsch'], ['Albacon Systemhaus GmbH', '6', '3.89', '

https://www.dnb.com/business-directory/company-profiles.webgo_gmbh.2d764e31c811ce3dfc551b55550b15e3.html
[['vleet GmbH', '3', '0.3', '82152', ' Planegg'], ['Emmet Software Labs GmbH & Co. KG', '23', '0.73', '32052', ' Herford'], ['kimeta GmbH', '61', '8.64', '64295', ' Darmstadt'], ['DataCo GmbH', '100', '5.9', '80335', ' MÜNchen'], ['Baremos IT Dienstleistungs GmbH', '3', '3', '10407', ' Berlin'], ['711media websolutions GmbH', '20', '1.97', '70178', ' Stuttgart'], ['clickworker GmbH', '20', 'missing', '45149', ' Essen'], ['AMPADA GmbH', '9', '1.32', '50765', ' KÖLn'], ['tradefinity GmbH', '28', 'DEC', '25474', ' Ellerbek'], ['UFIT AG', '13', '1.25', '67141', ' Neuhofen'], ['FORCAM GmbH', '95', '14.04', '88214', ' Ravensburg'], ['Müller und Richter Informationssysteme GmbH', '5', '0.59', '63571', ' Gelnhausen'], ['OTRS AG', '63', '8', '61440', ' Oberursel (Taunus)'], ['Conclusive Consulting GmbH', '8', '0.44', '64653', ' Lorsch'], ['Albacon Systemhaus GmbH', '6', '3.89', '18209', ' Ba

not in db
Schweers Informationstechnologie GmbH
not in db
oPen Software GmbH
not in db
None


TypeError: must be str, not NoneType